In [24]:
from psaw import PushshiftAPI                               #Importing wrapper library for reddit(Pushshift)
import datetime as dt                                       #Importing library for date management
import pandas as pd                                         #Importing library for data manipulation in python
import matplotlib.pyplot as plt                             #Importing library for creating interactive visualizations in Python
from pprint import pprint         

In [25]:
pd.set_option("display.max_columns", None)                  #Configuration for pandas to show all columns on dataframe
api = PushshiftAPI()   

In [26]:
# # with subreddit

# def data_prep_posts(subreddit, start_time, end_time, filters, keyword, limit):
#     if(len(filters) == 0):
#         filters = ['id', 'author', 'created_utc',
#                    'domain', 'url',
#                    'title', 'num_comments']                 #We set by default some columns that will be useful for data analysis

#     posts = list(api.search_submissions(
#         subreddit=subreddit,                                           #We set the subreddit we want to audit
#         after=start_time,                                   #Start date
#         before=end_time,                                    #End date
#         filter=filters,
#         q = keyword,
#         limit=limit))                                       #Max number of posts we wanto to recieve

#     return pd.DataFrame(posts)                              #Return dataframe for analysis

#no subreddit

def data_prep_posts(start_time, end_time, filters, keyword, limit):
    if(len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'domain', 'url',
                   'title', 'num_comments']                 #We set by default some columns that will be useful for data analysis

    posts = list(api.search_submissions(                                          #We set the subreddit we want to audit
        after=start_time,                                   #Start date
        before=end_time,                                    #End date
        filter=filters,
        q = keyword,
        limit=limit))                                       #Max number of posts we wanto to recieve

    return pd.DataFrame(posts)                              #Return dataframe for analysis


In [27]:
"""FOR COMMENTS"""
def data_prep_comments(term, start_time, end_time, filters, limit):
    if (len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'body', 'permalink', 'subreddit']
                   #We set by default some usefull columns 

    comments = list(api.search_comments(
        q=term,                 #Subreddit we want to audit
        after=start_time,       #Start date
        before=end_time,        #End date
        filter=filters,         #Column names we want to retrieve
        limit=limit))           #Max number of comments
    return pd.DataFrame(comments) #Return dataframe for analysis

In [28]:
def count_posts_per_date(df_p, title, xlabel, ylabel):
    df_p.groupby([df_p.datetime.dt.date]).count().plot(y='id', rot=45, kind='bar', label='Posts')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [29]:

"""
BEGIN - FUNCTIONS
"""
###Function to plot the number of posts per day on the specified subreddit
def count_posts_per_date(df_p, title, xlabel, ylabel):
    df_p.groupby([df_p.datetime.dt.date]).count().plot(y='id', rot=45, kind='bar', label='Posts')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

###Function to plot the mean of comments per day on the specified subreddit
def mean_comments_per_date(df_p, title, xlabel, ylabel):
    df_p.groupby([df_p.datetime.dt.date]).mean().plot(y='num_comments', rot=45, kind='line', label='Comments')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

###Function to plot the most active users on the subreddit
def most_active_author(df_p, title, xlabel, ylabel, limit):
    df_p.groupby([df_p.author]).count()['id'].nlargest(limit).sort_values(ascending=True).plot(y='id', rot=45, kind='barh', label='Users')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

###Function to que the orgin form the crossposting
def get_posts_orign(df_p, title, xlabel, ylabel, limit, subreddit):
    domains = df_p[(df_p.domain != 'reddit.com') & (df_p.domain != f'self.{subreddit}') & (df_p.domain != 'i.redd.it')]
    domains.groupby(by='domain').count()['id'].nlargest(limit).sort_values(ascending=True).plot(kind='barh', rot=45, x='domain', label='# of posts', legend=True, figsize=(8,13))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

###Gets most active subrredits according to search term
def get_subreddits(df_p, title, xlabel, ylabel, limit):
    df_p.groupby(by='subreddit').count()['id'].nlargest(limit).sort_values(ascending=True).plot(kind='barh', x='subreddit', label='Subreddit', legend=True, figsize=(8,13))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


"""
END - FUNCTIONS
"""

'\nEND - FUNCTIONS\n'

In [30]:
#with no subreddit

def main():                   #Name of the subreddit we are auditing
    start_time = int(dt.datetime(2022, 11, 1).timestamp())  #We define the starting date for our search
    end_time = int(dt.datetime(2022,12, 1).timestamp())   #We define the ending date for our search
    filters = [] 
    keyword = "AAPL"
    limit = 3000                                         #Number of elelemts we want to recieve

    """Here we are going to get subreddits for a brief analysis"""
    df_p = data_prep_posts(start_time,
                         end_time,filters,keyword,limit)           #Call function for dataframe creation of comments

    df_p['datetime'] = df_p['created_utc'].map(
        lambda t: dt.datetime.fromtimestamp(t))
    df_p = df_p.drop('created_utc', axis=1)                #Drop the column on timestamp format
    df_p = df_p.sort_values(by='datetime')                 #Sort the Row by datetime
    df_p["datetime"] = pd.to_datetime(df_p["datetime"])    #Convert timestamp format to datetime for data analysis


    df_p.to_csv(f'nov-{keyword}_dataset_all_posts.csv', sep=',', # Save the dataset on a csv file for future analysis
                header=True, index=False, columns=[
            'id', 'author', 'datetime', 'domain',
            'url', 'title', 'num_comments'
        ])

#     count_posts_per_date(df_p, 'Post per day', 'Days',     #Function to plot the number of posts per day on the specified subreddit
#                          'posts')
#     mean_comments_per_date(df_p,                           #Function to plot the mean of comments per day on the specified subreddit
#                            'Average comments per day',
#                            'Days', 'comments')
#     most_active_author(df_p, 'Most active users',          #Function to plot the most active users on the subreddit
#                        'Posts', 'Users', 10)
#     get_posts_orign(df_p, 'Origin of crosspostings',       #Function to que the orgin form the crossposting
#                     'Crossposts', 'Origins', 10,
#                     subreddit)

if __name__== "__main__" : main()

In [31]:
# def main():
#     subreddit = "wallstreetbets"                           #Name of the subreddit we are auditing
#     start_time = int(dt.datetime(2022, 11, 1).timestamp())  #We define the starting date for our search
#     end_time = int(dt.datetime(2022,12, 1).timestamp())   #We define the ending date for our search
#     filters = [] 
#     keyword = "AAPL"
#     limit = 3000                                         #Number of elelemts we want to recieve

#     """Here we are going to get subreddits for a brief analysis"""
#     df_p = data_prep_posts(subreddit, start_time,
#                          end_time,filters,keyword,limit)           #Call function for dataframe creation of comments

#     df_p['datetime'] = df_p['created_utc'].map(
#         lambda t: dt.datetime.fromtimestamp(t))
#     df_p = df_p.drop('created_utc', axis=1)                #Drop the column on timestamp format
#     df_p = df_p.sort_values(by='datetime')                 #Sort the Row by datetime
#     df_p["datetime"] = pd.to_datetime(df_p["datetime"])    #Convert timestamp format to datetime for data analysis


#     df_p.to_csv(f'nov-{keyword}_dataset_{subreddit}_posts.csv', sep=',', # Save the dataset on a csv file for future analysis
#                 header=True, index=False, columns=[
#             'id', 'author', 'datetime', 'domain',
#             'url', 'title', 'num_comments'
#         ])

#     count_posts_per_date(df_p, 'Post per day', 'Days',     #Function to plot the number of posts per day on the specified subreddit
#                          'posts')
#     mean_comments_per_date(df_p,                           #Function to plot the mean of comments per day on the specified subreddit
#                            'Average comments per day',
#                            'Days', 'comments')
#     most_active_author(df_p, 'Most active users',          #Function to plot the most active users on the subreddit
#                        'Posts', 'Users', 10)
#     get_posts_orign(df_p, 'Origin of crosspostings',       #Function to que the orgin form the crossposting
#                     'Crossposts', 'Origins', 10,
#                     subreddit)

# if __name__== "__main__" : main()